In [1]:
from collections import deque

class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list

    def get_neighbors(self, v):
        return self.adjacency_list[v]

    def h(self, n):
        H = {
            'A': 3,
            'B': 4,
            'C': 2,
            'D': 6,
            'G': 0,
            'S': 5
        }
        return H[n]

    def a_star_algorithm(self, start_node, stop_node):
        open_list = set([start_node])
        closed_list = set()
        g = {start_node: 0}
        parents = {start_node: start_node}

        while open_list:
            n = None

            for v in open_list:
                if n is None or g[v] + self.h(v) < g[n] + self.h(n):
                    n = v

            if n is None:
                print('Path does not exist!')
                return None

            if n == stop_node:
                reconst_path = []
                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]
                reconst_path.append(start_node)
                reconst_path.reverse()
                print('Path found: {}'.format(reconst_path))
                return reconst_path

            for (m, weight) in self.get_neighbors(n):
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n
                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            open_list.remove(n)
            closed_list.add(n)

        print('Path does not exist!')
        return None


In [2]:
adjacency_list = {
    'A': [('B', 2), ('C', 1)],
    'B': [('D', 5)],
    'C': [('D', 3), ('G', 4)],
    'D': [('G', 2)],
    'S': [('A', 1), ('G', 10)]
}

graph1 = Graph(adjacency_list)
graph1.a_star_algorithm('S', 'G')


Path found: ['S', 'A', 'C', 'G']


['S', 'A', 'C', 'G']

In [1]:
import numpy as np
from copy import deepcopy
import time

In [2]:
def misplaced_tiles(puzzle, goal):
    return sum([1 if puzzle[i] != goal[i] and puzzle[i] != 0 else 0 for i in range(9)])

In [3]:
def bestsolution(state):
    bestsol = []
    current = len(state) - 1
    while current != -1:
        bestsol.insert(0, np.array(state[current]['puzzle']).reshape(3, 3))
        current = state[current]['parent']
    return bestsol

In [4]:
def evaluvate_misplaced(puzzle, goal):
    steps = [
        ('up', [0, 1, 2], -3),
        ('down', [6, 7, 8], 3),
        ('left', [0, 3, 6], -1),
        ('right', [2, 5, 8], 1)
    ]
    
    dtstate = [('puzzle', list), ('parent', int), ('gn', int), ('hn', int)]
    dtpriority = [('position', int), ('fn', int)]

    parent = -1
    gn = 0
    hn = misplaced_tiles(puzzle, goal)
    state = np.array([(puzzle, parent, gn, hn)], dtype=dtstate)
    priority = np.array([(0, hn)], dtype=dtpriority)

    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])
        position, fn = priority[0]
        priority = np.delete(priority, 0, 0)

        puzzle, parent, gn, hn = state[position]
        puzzle = list(puzzle)
        blank = puzzle.index(0)
        gn = gn + 1

        for move, pos, offset in steps:
            if blank in pos:
                continue
            new_puzzle = puzzle.copy()
            new_puzzle[blank], new_puzzle[blank + offset] = new_puzzle[blank + offset], new_puzzle[blank]
            
            if any((np.array_equal(np.array(s['puzzle']), np.array(new_puzzle))) for s in state):
                continue
            
            hn = misplaced_tiles(new_puzzle, goal)
            state = np.append(state, np.array([(new_puzzle, position, gn, hn)], dtype=dtstate))
            fn = gn + hn
            priority = np.append(priority, np.array([(len(state) - 1, fn)], dtype=dtpriority))

            if new_puzzle == goal:
                print("The 8 puzzle is solvable.\n")
                return state

        if len(priority) == 0:
            print("The 8 puzzle is unsolvable.\n")
            return state

In [5]:
puzzle = [2, 8, 3,
          1, 6, 4,
          7, 0, 5]

goal = [1, 2, 3,
        8, 0, 4,
        7, 6, 5]

state = evaluvate_misplaced(puzzle, goal)
bestpath = bestsolution(state)

for i, step in enumerate(bestpath):
    print(f"\nStep {i}:\n{step}")

print("\nTotal Steps to reach goal:", len(bestpath) - 1)

The 8 puzzle is solvable.


Step 0:
[[2 8 3]
 [1 6 4]
 [7 0 5]]

Step 1:
[[2 8 3]
 [1 0 4]
 [7 6 5]]

Step 2:
[[2 0 3]
 [1 8 4]
 [7 6 5]]

Step 3:
[[0 2 3]
 [1 8 4]
 [7 6 5]]

Step 4:
[[1 2 3]
 [0 8 4]
 [7 6 5]]

Step 5:
[[1 2 3]
 [8 0 4]
 [7 6 5]]

Total Steps to reach goal: 5
